## Import

In [ ]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import importlib as il
from hypso import Hypso1, Hypso2
import src.deh as deh
import copy
import gc
il.reload(deh)
sys.path.append(os.path.abspath("D:/Hierarchical Unmixing Label"))
HYPSO_WIDTH=598
HYPSO_HEIGHT=1092
HYPSO_BANDS=112 # using bands 6:118 of 120

## Functions

In [ ]:
#Functions
def plot_cm_rgb_composite(cm, title="", red_band_index=69, green_band_index=46, blue_band_index=26, aspect=0.1, figsize=(10, 10), height=1092, contrast_enhancement=False):
    """
    Create and plot an RGB composite image from a hyperspectral cube.
    
    Parameters:
    - cm: Input hyperspectral data cube or array (can be flattened (h*w, n_bands) or 3D (h, w, n_bands))
    - title: Optional title for the plot
    - red_band_index: Index of the band to use for red channel (default: 69)
    - green_band_index: Index of the band to use for green channel (default: 46)
    - blue_band_index: Index of the band to use for blue channel (default: 26)
    - aspect: Aspect ratio for the plot (default: 0.1)
    - figsize: Figure size as tuple (width, height) in inches (default: (10, 10))
    - height: Height of the image when reshaping from flattened data (default: 1092)
    
    Returns:
    - rgb_image: The processed RGB image
    """
    # Check if input is already a 3D cube or needs reshaping
    if len(cm.shape) == 2:  # Flattened data (h*w, n_bands)
        width = cm.shape[0] // height
        data_cube = cm.reshape(width, height, cm.shape[1])
        data_cube = np.transpose(data_cube, (0, 1, 2))  # Ensure correct orientation
    elif len(cm.shape) == 3:  # Already a cube (h, w, n_bands)
        data_cube = cm
    else:
        raise ValueError(f"Unexpected input shape: {cm.shape}. Expected 2D or 3D array.")
    
    # Extract the specified bands for RGB channels
    red_band = data_cube[:, :, red_band_index]
    green_band = data_cube[:, :, green_band_index]
    blue_band = data_cube[:, :, blue_band_index]

    # Stack the bands to create an RGB image
    rgb_image = np.stack((red_band, green_band, blue_band), axis=-1)
    
    # Process data for better visualization
    # Replace NaN values with 0
    rgb_image[np.isnan(rgb_image)] = 0

    # Apply normalization to each channel
    for i in range(3):
        channel = rgb_image[:,:,i]
        
        # Always use min-max normalization regardless of value range
        min_val = np.nanmin(channel)
        max_val = np.nanmax(channel)
        
        if max_val > min_val:  # Avoid division by zero
            # Normalize to [0,1] range
            channel = (channel - min_val) / (max_val - min_val)
            
            # Apply contrast enhancement if requested
            if contrast_enhancement and np.any(channel > 0):
                # Only enhance contrast if we have enough non-zero values
                non_zero_values = channel[channel > 0]
                if len(non_zero_values) > 10:  # Arbitrary threshold
                    percentiles = np.nanpercentile(channel, [2, 98])
                    p_low, p_high = percentiles[0], percentiles[1]
                    if p_high > p_low:
                        channel = np.clip(channel, p_low, p_high)
                        channel = (channel - p_low) / (p_high - p_low)
            
            rgb_image[:,:,i] = channel
    
    # Final normalization and cleanup
    rgb_image = np.clip(rgb_image, 0, 1)
    
    # Rotate for proper orientation
    rgb_image = np.rot90(rgb_image)
    
    # Create and display the plot
    fig, ax = plt.subplots(1, 1, figsize=figsize)
    if title:
        plt.title(title)
    plt.imshow(rgb_image, aspect=aspect)
    plt.axis('off')  # Hide axes for cleaner visualization
    
    return rgb_image

def create_rgb_composite_from_satobj(satobj, title="", cube="L1A", red_band_index=69, green_band_index=46, blue_band_index=26, aspect=0.1, figsize=(10, 10), display=True, contrast_enhancement=False ):
    """
    Create an RGB composite image from a satellite object (satobj).

    Parameters:
    - satobj: An instance of the satellite object containing the image data.
    - red_band_index: The index of the band to use for the red channel (default: 69).
    - green_band_index: The index of the band to use for the green channel (default: 46).
    - blue_band_index: The index of the band to use for the blue channel (default: 26).
    - aspect: Aspect ratio for the plot (default: 0.1).
    - figsize: Figure size as tuple (width, height) in inches (default: (10, 10)).
    - display: Whether to display the image (default: True).
    - cube: Which data cube to use ("L1A", "L1B", "L1C", or "L1D") (default: "L1A").

    Returns:
    - rgb_image: A 3D numpy array representing the RGB composite image.
    """
    # Get the data cube from the satellite object
    if cube =="L1A":
        data_cube = satobj.l1a_cube  
    elif cube =="L1B":
        data_cube = satobj.l1b_cube  
    elif cube =="L1C":
        data_cube = satobj.l1c_cube 
    elif cube =="L1D":
        data_cube = satobj.l1d_cube
    else:
        print("cube not valid, defaulting tp L1A")
        data_cube = satobj.l1a_cube     
    
    # Extract the specified bands for RGB channels
    red_band = data_cube[:, :, red_band_index]
    green_band = data_cube[:, :, green_band_index]
    blue_band = data_cube[:, :, blue_band_index]

    # Stack the bands to create an RGB image
    rgb_image = np.stack((red_band, green_band, blue_band), axis=-1)

    # Handle NaN values
    rgb_image[np.isnan(rgb_image)] = 0

    # Apply normalization to each channel
    for i in range(3):
        channel = rgb_image[:,:,i]
        
        # Always use min-max normalization regardless of value range
        min_val = np.nanmin(channel)
        max_val = np.nanmax(channel)
        
        if max_val > min_val:  # Avoid division by zero
            # Normalize to [0,1] range
            channel = (channel - min_val) / (max_val - min_val)
            
            # Apply contrast enhancement if requested
            if contrast_enhancement and np.any(channel > 0):
                # Only enhance contrast if we have enough non-zero values
                non_zero_values = channel[channel > 0]
                if len(non_zero_values) > 10:  # Arbitrary threshold
                    percentiles = np.nanpercentile(channel, [2, 98])
                    p_low, p_high = percentiles[0], percentiles[1]
                    if p_high > p_low:
                        channel = np.clip(channel, p_low, p_high)
                        channel = (channel - p_low) / (p_high - p_low)
            
            rgb_image[:,:,i] = channel
    
    # Final normalization and cleanup
    rgb_image = np.clip(rgb_image, 0, 1)
    
    # Create properly oriented image for display
    display_image = np.rot90(rgb_image)
    
    # Display the image if requested
    if display:
        fig, ax = plt.subplots(figsize=figsize)
        ax.imshow(display_image, aspect=aspect, interpolation='nearest')
        ax.set_title(title+" "+cube)
        ax.axis('off')
        plt.show()

    return rgb_image

def plot_DEH_overlay(DEH_input, rgb_image, node, opacity=0.5):
    """
    Plot and return the overlay image of DEH figure on top of an RGB image with controllable opacity.
    Non-zero DEH values will be shown with the specified opacity, while zero values will be fully transparent.
    Both normal and inverse overlays will be shown side by side.

    Parameters:
    - DEH_input: The DEH input object containing the node data
    - rgb_image: The RGB image to overlay the DEH figure on, shape (w, h, 3)
    - node: The node to visualize
    - opacity: The opacity of the DEH figure where values are non-zero (0.0 to 1.0)

    Returns:
    - overlay_image: The RGB image with the DEH figure overlaid
    """
    deh_figure = DEH_input.nodes[node].map.reshape(DEH_input.plot_size)
    
    # Normalize both the DEH figure and the RGB image to the range [0, 1]
    deh_figure_normalized = (deh_figure - np.min(deh_figure)) / (np.max(deh_figure) - np.min(deh_figure))
    rgb_image_normalized = (rgb_image - np.min(rgb_image)) / (np.max(rgb_image) - np.min(rgb_image))

    # Create a red overlay
    red_overlay = np.ones_like(deh_figure_normalized)[:, :, np.newaxis] * [1, 0, 0]  # Red color

    # Create normal overlay
    overlay_image = rgb_image_normalized.copy()
    non_zero_mask = deh_figure != 0
    overlay_image[non_zero_mask] = (rgb_image_normalized[non_zero_mask] * (1 - opacity) + 
                                  red_overlay[non_zero_mask] * opacity * deh_figure_normalized[non_zero_mask, np.newaxis])

    # Create inverse overlay
    inverse_overlay = rgb_image_normalized.copy()
    inverse_mask = deh_figure == 0
    inverse_overlay[inverse_mask] = (rgb_image_normalized[inverse_mask] * (1 - opacity) + 
                                   red_overlay[inverse_mask] * opacity)

    # Plot both overlays side by side
    fig, axs = plt.subplots(1, 2, figsize=(50, 200), gridspec_kw={'wspace': 0.1, 'hspace': 0.1})
    
    axs[0].imshow(np.rot90(overlay_image), aspect=DEH_input.plot_aspect, vmin=0, vmax=1, interpolation='bicubic')
    axs[0].set_title(f"Node: {node} - Original")
    axs[0].axis('off')

    axs[1].imshow(np.rot90(inverse_overlay), aspect=DEH_input.plot_aspect, vmin=0, vmax=1, interpolation='bicubic')
    axs[1].set_title(f"Node: {node} - Inverse")
    axs[1].axis('off')

    plt.show()

    return overlay_image

def DEH_init_stabelize(DEH_input, cm_input, saturated_input, save_name, n_endmembers=8, n_runs=20, verbose=True):
    #Init DEH
    DEH_input.splitting_size=1000
    DEH_input.max_depth=2
    DEH_input.max_iter=5
    DEH_input.max_nodes=2


    cube=cm_input.reshape(-1,1092,112)
    #these set the size of the image to be plotted
    DEH_input.plot_size = (cube.shape[0],cube.shape[1])
    DEH_input.plot_aspect = 0.1

    # these set the normaliztion on the data, but right now it is turned off and replaced with a nearest-neighbor noise estimate
    DEH_input.weight_power=0
    DEH_input.eps = 0.00
    #dehx.wf = lambda x: (np.sum(x**2, axis=-1))**(-1/2+1/(2+dehx.eps))

    #this switches between Archetype Analysis and Pure Pixel Analysis
    DEH_input.aa = False

    #this is currently replaced by "mpp_tol" in the training code
    DEH_input.mixed_pix = 0

    # this is the prefactor on the L2 regularization
    DEH_input.reg=0

    # if this is anything other than zero, it introduces L2 regularization on the trained weights 
    DEH_input.set_mu(0)

    # this turns on a sort of protection to keep endmembers from vanishing. Generally unneeded with new training
    DEH_input.use_bonus_boost = False

    # turns on normalization of the data. Generally always used
    DEH_input.use_norm(True)

    # if greater than 1, turns on a sort of spectral sampling with PAA. Occasionally gives good performance
    # but I generally keep it at 1 to turn it off
    DEH_input.PAA_backcount = 1

    # changes the gradient descent learning rate as a proportion of the optimal rate. I recommend keeping it at 1
    DEH_input.a_speed= 1.0

    #Verbose print
    if verbose:
        print("cm_input.shape: ", cm_input.shape)
        print("cube.shape: ", cube.shape)
        print("DEH_input.plot_size", DEH_input.plot_size)
        print("saturated_input.shape", saturated_input.shape)
        print("Quick nn")
    
    #random Init
    DEH_input.neighbors = deh.quick_nn(cm_input.reshape(DEH_input.plot_size + (-1,)), k_size=1).flatten()
    if verbose: 
        print("Setting neighbor weights")
    DEH_input.set_neighbor_weights(cm_input)
    if verbose:
        print("Random init")
    DEH_input.random_init(cm_input, n_endmembers)
    if verbose:
        print("Simple predict")
    deh_pred=DEH_input.simple_predict(cm_input)
    if verbose:
        print("Getting trimmed network")

    #Stabelized
    DEH_stabelized=DEH_input.get_trimmed_network(DEH_input.nodes) #Can replace with copy of DEH_input instead of get trimmed network since it acceptes all nodes
    if verbose:
        print("Quick nn")
    DEH_stabelized.neighbors = deh.quick_nn(cm_input.reshape(DEH_stabelized.plot_size + (-1,)), k_size=1).flatten()
    if verbose:
        print("Setting neighbor weights")

    DEH_stabelized.set_neighbor_weights(cm_input)
    if verbose:
        print("DEH_stabelized.full_weights.shape",DEH_stabelized.full_weights.shape)
    #saturated_input = np.asarray(saturated_input, dtype=bool)
    DEH_stabelized.full_weights[saturated_input]=0
    DEH_stabelized.PAA_backcount=1


    if verbose:
        print("Accepted network stablization")
    DEH_stabelized.accepted_network_stablization(cm_input, n_runs=n_runs, n_pts=(1000,10000), obj_record=(), sampling_points=(), mpp_tol=0.2, step_delta=0.01, reg_max=0.2, name=save_name)

    return DEH_input, DEH_stabelized

def plot_level_and_overlay(save_file_path, cm_input, plot_only_last_level=False, binarize=False, opacity=0.5):
    DEH_temp = deh.DEH(no_negative_residuals=True)
    DEH_temp.load(save_file_path)
    cube_image=cm_input.reshape(-1,1092,112)

    DEH_temp.plot_size = (cube_image.shape[0],cube_image.shape[1])
    DEH_temp.verbose = False
    deh_predicted = DEH_temp.simple_predict(cm_input)
    rgb_image=plot_cm_rgb_composite(cm_input)
    nodes=DEH_temp.nodes
    num_levels = max(len(node) for node in DEH_temp.nodes)
    levels_to_plot = [num_levels] if plot_only_last_level else range(1, num_levels + 1)
    if binarize:
        DEH_temp.binarize_lmdas()
        DEH_temp.lmda_2_map()
    
    for level in levels_to_plot:
        DEH_temp.display_level(level)
        for node in nodes:
            if len(node) == level:
                overlay_image = plot_DEH_overlay(DEH_temp, rgb_image, node=node, opacity=opacity)

def plot_spectra_for_level(save_file_path, cm_input, level_input=""):
    DEH_temp = deh.DEH(no_negative_residuals=True)
    DEH_temp.load(save_file_path)
    cube_image=cm_input.reshape(-1,1092,112)

    DEH_temp.plot_size = (cube_image.shape[0],cube_image.shape[1])
    DEH_temp.simple_predict(cm_input)
    nodes = DEH_temp.nodes
    num_levels = max(len(node) for node in DEH_temp.nodes)
    
    # If level_input is empty or invalid, use the last level
    if level_input == "" or not isinstance(level_input, int) or level_input > num_levels or level_input < 1:
        level = num_levels
    else:
        level = level_input
        
    # Get nodes at specified level
    level_nodes = [node for node in nodes if len(node) == level]
    print("using level", level, "with", len(level_nodes), "nodes")
    DEH_temp.display_spectra(level_nodes)

def binarize_DEH(save_file_path, cm_input):
    DEH_temp = deh.DEH(no_negative_residuals=True)
    DEH_temp.load(save_file_path)
    cube_image=cm_input.reshape(-1,1092,112)

    DEH_temp.plot_size = (cube_image.shape[0],cube_image.shape[1])
    DEH_temp.simple_predict(cm_input)
    DEH_binarized=copy.deepcopy(DEH_temp)
    DEH_binarized.binarize_lmdas()
    DEH_binarized.lmda_2_map()

    return DEH_binarized

def load_DEH(save_file_path):
    DEH_temp = deh.DEH(no_negative_residuals=True)
    DEH_temp.load(save_file_path)
    return DEH_temp

def save_binarized_labels_to_npy(deh_path, data, save_path=None):
    """
    Save binarized labels from DEH model to a .npy file.
    Labels are stored as a dictionary with node names as keys and binary arrays as values.
    
    Args:
        deh_model: The DEH model with binarized labels
        data: Input data used for prediction 
        output_path: Path to save the output .npy file
    """
    import numpy as np
    deh_binarized=deh.DEH(no_negative_residuals=True)
    deh_binarized.load(deh_path)
    # Get predictions and binarize
    print(data.shape)
    cube=data.reshape(-1,1092,112)
    deh_binarized.plot_size=(cube.shape[0],cube.shape[1])
    print(deh_binarized.plot_size)
    deh_binarized.simple_predict(data)
    deh_binarized.binarize_lmdas()
    deh_binarized.lmda_2_map()
    
    # Create dictionary to store labels
    labels_dict = {}
    
    # Store binary labels for each node
    for node in deh_binarized.nodes:
        # Get binary labels from node.map which already contains the binarized values
        labels = deh_binarized.nodes[node].map.flatten()
        labels_dict[node] = labels
        
    # Save dictionary to .npy file
    # Use provided savepath if available, otherwise create from deh_path
    if save_path is not None:
        output_path = save_path
    else:
        # Create output path based on deh_path
        # Replace file extension and add '_labels' to the filename
        base_path = os.path.splitext(deh_path)[0]
        output_path = base_path + '_labels.npy'
    
    print(f"Saving binarized labels to: {output_path}")
    np.save(output_path, labels_dict)

def plot_level_labels(npy_file, level=None):
    """
    Plot binary labels for nodes at specified level, highlighting overlapping regions.
    
    Args:
        npy_file: Path to .npy file containing binary labels
        level: Level of nodes to plot (based on key length). If None, uses max level.
    """
    # Load labels
    labels = np.load(npy_file, allow_pickle=True).item()
    
    # Get all keys and their lengths
    key_lengths = [len(k) for k in labels.keys()]
    max_level = max(key_lengths)
    
    # If level not specified or invalid, use max level
    if level is None or level > max_level:
        level = max_level
        
    # Get keys for specified level
    level_keys = [k for k in labels.keys() if len(k) == level]
    
    if not level_keys:
        print(f"No nodes found at level {level}")
        return
        
    # Setup plot
    fig, ax = plt.subplots(figsize=(10, 10))
    
    # Create combined array to track overlaps
    first_key = level_keys[0]
    combined_labels = np.zeros(labels[first_key].reshape(-1, 1092).shape)
    
    # First pass - count overlaps
    for key in level_keys:
        plot_label = labels[key].reshape(-1, 1092)
        combined_labels += plot_label
    
    # Plot each node's labels
    for i, key in enumerate(level_keys):
        plot_label = labels[key].reshape(-1, 1092)
        # Create custom colormap for this node
        colors = [(1,1,1,0), plt.cm.rainbow(i/len(level_keys))]  # Transparent white to color
        node_cmap = plt.matplotlib.colors.LinearSegmentedColormap.from_list(f'custom_{i}', colors)
        alpha = 0.5 if np.any(combined_labels > 1) else 1.0
        plt.imshow(np.rot90(plot_label), aspect=0.1, alpha=alpha, cmap=node_cmap)
    
    # Plot overlaps with a different color if they exist
    if np.any(combined_labels > 1):
        overlap_mask = combined_labels > 1
        overlap_display = np.rot90(overlap_mask.astype(float))
        plt.imshow(overlap_display, aspect=0.1, alpha=0.7, cmap='Reds', 
                   label='Overlapping Regions')
    
    plt.title(f'Binary Labels for Level {level} Nodes')
    
    # Add legend
    legend_elements = [plt.Rectangle((0,0),1,1, facecolor=plt.cm.rainbow(i/len(level_keys))) 
                      for i in range(len(level_keys))]
    if np.any(combined_labels > 1):
        legend_elements.append(plt.Rectangle((0,0),1,1, facecolor='red', alpha=0.7))
        legend_labels = list(level_keys) + ['Overlaps']
    else:
        legend_labels = list(level_keys)
    ax.legend(legend_elements, legend_labels, loc='center left', bbox_to_anchor=(1, 0.5))
    
    plt.tight_layout()
    plt.show()

def find_top_endmembers(deh_obj, n=8, plot=False, verbose=False):
    """
    Find the n endmembers with the highest abundance in the DEH object.
    
    Args:
        deh_obj: The DEH object to analyze
        n: Number of top endmembers to return (default: 8)
        plot: Whether to plot the top endmembers (default: True)
        verbose: Whether to print detailed information (default: False)
        
    Returns:
        Dictionary of top endmembers with their node IDs and abundance values
    """
    # Get the deepest level nodes (endmembers)
    max_level = max(len(node_id) for node_id in deh_obj.nodes.keys())
    endmember_nodes = {node_id: node for node_id, node in deh_obj.nodes.items() if len(node_id) == max_level}
    
    if verbose:
        print(f"Total endmembers at level {max_level}: {len(endmember_nodes)}")
    
    # Calculate abundance for each endmember
    abundances = {}
    for node_id, node in endmember_nodes.items():
        if hasattr(node, 'map'):
            abundances[node_id] = deh_obj.nodes[node_id].map.sum()
        elif hasattr(node, 'abundance'):
            abundances[node_id] = node.abundance
        elif hasattr(node, 'weight'):
            abundances[node_id] = node.weight
        else:
            if verbose:
                print(f"Warning: Node {node_id} has no map, abundance or weight attribute")
    
    # Sort by abundance and get top n
    top_endmembers = dict(sorted(abundances.items(), key=lambda x: x[1], reverse=True)[:n])
    
    # Print results if verbose
    if verbose:
        print(f"\nTop {n} endmembers by abundance:")
        for i, (node_id, abundance) in enumerate(top_endmembers.items(), 1):
            print(f"{i}. Node {node_id}: Abundance = {abundance:.4f}")
    
    return top_endmembers

def print_nodes_by_depth(deh_obj):
    """
    Print all nodes in the DEH object organized by depth.
    
    Args:
        deh_obj: The DEH object containing nodes
    """
    # Get all nodes
    all_nodes = list(deh_obj.nodes.keys())
    
    # Group nodes by depth
    nodes_by_depth = {}
    for node_id in all_nodes:
        depth = len(node_id)
        if depth not in nodes_by_depth:
            nodes_by_depth[depth] = []
        nodes_by_depth[depth].append(node_id)
    
    # Print nodes by depth
    print("Nodes by depth:")
    for depth in sorted(nodes_by_depth.keys()):
        node_count = len(nodes_by_depth[depth])
        print(f"Depth {depth}: {node_count} nodes")
        # Print the first 10 nodes at this depth as examples
        example_nodes = nodes_by_depth[depth][:10]
        if example_nodes:
            print(f"  Examples: {', '.join(example_nodes)}")
        if node_count > 10:
            print(f"  ... and {node_count - 10} more")
        print()

def tree_walk(deh_obj, n=10, verbose=False):
    """
    Performs a tree walk starting from the root of a DEH object.
    The frontier expands by adding children of visited nodes.
    The next node to visit is selected based on the highest sum value.
    
    Parameters:
    -----------
    deh_obj : DEH object
        The hierarchical unmixing object to traverse
    n : int, default=10
        The number of nodes to have in the frontier before stopping
        
    Returns:
    --------
    list
        The nodes in the frontier when the threshold is reached
    """
    if verbose:
        print("Tree walking...")
    # Initialize with root node
    visited = []
    frontier = ['']  # Start with root node
    
    # Continue until frontier reaches desired size
    while len(frontier) < n:
        if not frontier:
            break
            
        # Find node with highest sum in frontier
        max_sum = -float('inf')
        max_node = None
        max_idx = -1
        
        for i, node_id in enumerate(frontier):
            node_sum = deh_obj.nodes[node_id].map.sum()
            if node_sum > max_sum:
                max_sum = node_sum
                max_node = node_id
                max_idx = i
        
        # Remove the selected node from frontier and add to visited
        frontier.pop(max_idx)
        visited.append(max_node)
        
        # Add children to frontier
        # Children of node 'x' are 'x0' and 'x1'
        left_child = max_node + '0'
        right_child = max_node + '1'
        
        # Check if children exist in the model
        if left_child in deh_obj.nodes:
            frontier.append(left_child)
        if right_child in deh_obj.nodes:
            frontier.append(right_child)
    
    # Calculate sum of all nodes in frontier
    frontier_sum = 0
    for node_id in frontier:
        frontier_sum += deh_obj.nodes[node_id].map.sum()
    
    # Calculate root node sum for comparison
    root_sum = deh_obj.nodes[''].map.sum()
    
    # Print results
    if verbose:
        print(f"Frontier nodes when threshold of {n} nodes reached:")
        for node_id in frontier:
            level = len(node_id)  # Level is determined by the length of the node_id
            print(f"Node: {node_id}, Level: {level}, Sum: {deh_obj.nodes[node_id].map.sum():.4f}")
        print(f"Total sum of frontier nodes: {frontier_sum:.4f}")
        print(f"Root node sum: {root_sum:.4f}")
        print(f"Difference: {root_sum - frontier_sum:.4f}")
    
    return frontier

def prune_deh_obj(deh_obj, node_list):
    """
    Prune a DEH object by keeping only the nodes in the given list and their ancestors,
    after extending all nodes to the same depth.
    
    Parameters:
    -----------
    deh_obj : DEH object
        The DEH object to prune
    node_list : list
        List of node IDs to keep (frontier nodes)
    
    Returns:
    --------
    pruned_deh_obj : DEH object
        The pruned DEH object
    """
    
    # Find the maximum depth of nodes in the node_list
    max_depth = max(len(node_id) for node_id in node_list)
    
    # Extend all nodes to the same depth by adding '0's
    extended_node_list = []
    for node_id in node_list:
        if len(node_id) < max_depth:
            # Extend the node_id by adding '0's until it reaches max_depth
            extended_node_id = node_id + '0' * (max_depth - len(node_id))
            extended_node_list.append(extended_node_id)
        else:
            extended_node_list.append(node_id)
    
    # Create a list of all nodes in the DEH object
    all_nodes = list(deh_obj.nodes.keys())
    
    # Create a set of nodes to keep (extended frontier nodes and their ancestors)
    nodes_to_keep = set(extended_node_list)
    
    # Add ancestors of frontier nodes to the keep list
    for node_id in extended_node_list:
        # Add all prefixes of the node_id (ancestors)
        for i in range(len(node_id)):
            nodes_to_keep.add(node_id[:i])
    
    # Add the root node
    nodes_to_keep.add('')
    
    # Create a list of nodes to delete
    nodes_to_delete = []
    for node_id in all_nodes:
        if node_id not in nodes_to_keep:
            # Check if any ancestor is already in the delete list
            # If so, we don't need to add this node as it will be deleted with its ancestor
            ancestor_in_delete_list = False
            for i in range(1, len(node_id)):
                if node_id[:i] in nodes_to_delete:
                    ancestor_in_delete_list = True
                    break
            
            if not ancestor_in_delete_list:
                nodes_to_delete.append(node_id)
    
    # Sort nodes to delete by depth (delete deepest nodes first)
    nodes_to_delete.sort(key=len, reverse=True)
    
    # Delete the nodes one by one
    deleted_count = 0
    for node_id in nodes_to_delete:
        try:
            if node_id in deh_obj.nodes:  # Check if node still exists
                # Delete the node (this will delete the node and its descendants)
                deh_obj.delete_node(node_id)
                deleted_count += 1
        except KeyError as e:
            # Node might have been already deleted
            print(f"Note: Node {node_id} was already deleted or caused error: {e}")
            continue
    
    print(f"Pruned {deleted_count} nodes and its decendants from the DEH object")
    print(f"Extended node list to depth {max_depth}: {extended_node_list}")

def train_random_DEH(image_data,endmembers, save_name, verbose=False):
    print("Training Random DEH...")
    DEH_random= deh.DEH(no_negative_residuals=True)
    #Init DEH
    DEH_random.splitting_size=1000
    DEH_random.max_depth=2
    DEH_random.max_iter=5
    DEH_random.max_nodes=2


    cube=image_data.reshape(-1,HYPSO_HEIGHT,112)
    #these set the size of the image to be plotted
    DEH_random.plot_size = (cube.shape[0],cube.shape[1])
    DEH_random.plot_aspect = 0.1

    # these set the normaliztion on the data, but right now it is turned off and replaced with a nearest-neighbor noise estimate
    DEH_random.weight_power=0
    DEH_random.eps = 0.00
    #dehx.wf = lambda x: (np.sum(x**2, axis=-1))**(-1/2+1/(2+dehx.eps))

    #this switches between Archetype Analysis and Pure Pixel Analysis
    DEH_random.aa = False

    #this is currently replaced by "mpp_tol" in the training code
    DEH_random.mixed_pix = 0

    # this is the prefactor on the L2 regularization
    DEH_random.reg=0

    # if this is anything other than zero, it introduces L2 regularization on the trained weights 
    DEH_random.set_mu(0)

    # this turns on a sort of protection to keep endmembers from vanishing. Generally unneeded with new training
    DEH_random.use_bonus_boost = False

    # turns on normalization of the data. Generally always used
    DEH_random.use_norm(True)

    # if greater than 1, turns on a sort of spectral sampling with PAA. Occasionally gives good performance
    # but I generally keep it at 1 to turn it off
    DEH_random.PAA_backcount = 1

    # changes the gradient descent learning rate as a proportion of the optimal rate. I recommend keeping it at 1
    DEH_random.a_speed= 1.0

    #Verbose print
    if verbose:
        print("cm_input.shape: ", image_data.shape)
        print("cube.shape: ", cube.shape)
        print("DEH_random.plot_size", DEH_random.plot_size)
    #random Init
    if verbose:
        print("Quick nn")
    DEH_random.neighbors = deh.quick_nn(image_data.reshape(DEH_random.plot_size + (-1,)), k_size=1).flatten()
    if verbose: 
        print("Setting neighbor weights")
    DEH_random.set_neighbor_weights(image_data)
    if verbose:
        print("Random init")
    DEH_random.random_init(image_data, endmembers)
    if verbose:
        print("Simple predict")
    DEH_random_pred=DEH_random.simple_predict(image_data)
    DEH_random.save(save_name)
    print("Training Random DEH complete, saved to ", save_name)
    
def prune_DEH(DEH_path, image_data, endmembers, verbose=False):
    print("Pruning DEH...")
    DEH_prune=deh.DEH(no_negative_residuals=True)
    DEH_prune.load(DEH_path)
    DEH_prune.verbose = verbose
    DEH_prune_pred=DEH_prune.simple_predict(image_data)
    frontier_nodes = tree_walk(DEH_prune, n=endmembers, verbose=verbose)
    prune_deh_obj(DEH_prune, frontier_nodes)
    if verbose:
        print("\nNodes by depth:")
        print_nodes_by_depth(DEH_prune)
    # Update the save path to reflect pruned model with correct number of endmembers
    save_path_parts = DEH_path.split('_')
    for i, part in enumerate(save_path_parts):
        if 'end' in part:
            # Find parts containing 'end' (like '256end')
            prefix = part.split('end')[0]  # Get the part before 'end'
            suffix = part.split('end')[1] if len(part.split('end')) > 1 else ''  # Get the part after 'end'
            # Replace with new endmembers value
            save_path_parts[i] = str(endmembers) + 'end' + suffix
        if 'random' in part.lower():
            # Replace 'random' with 'pruned'
            save_path_parts[i] = part.lower().replace('random', 'pruned')
    
    # Reconstruct the save path
    DEH_path = '_'.join(save_path_parts)
    if verbose:
        print(f"Updated save path: {DEH_path}")
    DEH_prune.save(DEH_path)
    print("Pruning DEH complete, saved to ", DEH_path)
    return DEH_path

def stabelize_DEH(DEH_path, image_data, saturated_image_data, n_runs=10, step_delta=0.05, verbose=False):
    print("Stabelizing DEH....")
    DEH_stabelize=deh.DEH(no_negative_residuals=True)
    DEH_stabelize.load(DEH_path)
    DEH_stabelize.verbose = verbose
    if verbose:
        print("Simple predict")
    DEH_stabelize.simple_predict(image_data)
    
    if verbose:
        print("Quick nn")
    DEH_stabelize.neighbors = deh.quick_nn(image_data.reshape(DEH_stabelize.plot_size + (-1,)), k_size=1).flatten()
    
    if verbose:
        print("Setting neighbor weights")

    DEH_stabelize.set_neighbor_weights(image_data)
    if verbose:
        print("DEH_stabelized.full_weights.shape",DEH_stabelize.full_weights.shape)
    
    #saturated_input = np.asarray(saturated_input, dtype=bool)
    DEH_stabelize.full_weights[saturated_image_data]=0
    DEH_stabelize.PAA_backcount=1
    
    
    # Create a new save path by replacing the last part with 'stabelized'
    save_path_parts = DEH_path.split('_')
    
    # Get file extension if present
    if '.' in save_path_parts[-1]:
        last_part, extension = save_path_parts[-1].rsplit('.', 1)
        # Remove .h5 extension if present
        if extension.lower() == 'h5':
            save_path_parts[-1] = 'stabelized'
        else:
            save_path_parts[-1] = 'stabelized.' + extension
    else:
        save_path_parts[-1] = 'stabelized'
    
    # Reconstruct the save path
    save_name = '_'.join(save_path_parts)
    if verbose:
        print(f"Updated save path for stabilization: {save_name}")

    if verbose:
        print("Accepted network stablization")
    DEH_stabelize.accepted_network_stablization(image_data, n_runs=n_runs, n_pts=(1000,10000), obj_record=(), sampling_points=(), mpp_tol=0.2, step_delta=step_delta, reg_max=0.2, name=save_name)
    print("Stabelizing DEH complete")

def reset_deh_for_reinit(deh_model):
    """
    Reset a DEH model so it can be randomly initialized again.
    
    Parameters:
    -----------
    deh_model : DEH
        The DEH model to reset
        
    Returns:
    --------
    deh_model : DEH
        The reset DEH model ready for new initialization
    """
    # Clear the tree structure
    deh_model.nodes = {}
    deh_model.lmdas = {}
    deh_model.leaf_mean = {}
    deh_model.path_to_node = {}
    deh_model.node_to_path = {}
    deh_model.splitting_nodes = set()
    deh_model.weights = {}
    deh_model.intercepts = {}
    
    # Clear model states
    if hasattr(deh_model, 'models'):
        deh_model.models = {}
    
    # Clear cache and results
    deh_model.preds = None
    deh_model.path_map = None
    deh_model.obj = float('inf')
    
    # Reset key parameters (if you've modified these)
    # deh_model.aa = False  # Only reset if you want to change this
    # deh_model.use_norm(True)  # Only reset if you want to change this
    
    # Note: We're keeping neighbors and full_weights since they're 
    # dependent on the input data, not the model structure
    
    return deh_model

def memory_efficient_set_neighbor_weights(deh_obj, data, batch_size=10000):
    """Memory-efficient version of setting neighbor weights"""
    import numpy as np
    
    n_samples = data.shape[0]
    deh_obj.full_weights = np.ones(n_samples)
    
    # Process in batches
    for start_idx in range(0, n_samples, batch_size):
        end_idx = min(start_idx + batch_size, n_samples)
        batch_indices = np.arange(start_idx, end_idx)
        
        # Only process non-saturated data if relevant
        rel_data = batch_indices
        
        # Skip if no relevant data in this batch
        if len(rel_data) == 0:
            continue
        
        # Process this batch
        neighbor_indices = deh_obj.neighbors[rel_data]
        
        # Calculate differences and weights for this batch
        diffs = data[rel_data] - data[neighbor_indices]
        expected_errs = np.sum(diffs**2, axis=1)
        
        # Update weights
        deh_obj.full_weights[rel_data] = 1 / (np.sqrt(expected_errs) + np.mean(np.sqrt(expected_errs)))
    
    # Normalize weights
    if np.sum(deh_obj.full_weights) > 0:
        deh_obj.full_weights /= np.mean(deh_obj.full_weights[deh_obj.full_weights > 0])
    
    return deh_obj

## Load data

### Old TOA DATA

In [ ]:
cm_combined_loaded = np.load('D:\Hierarchical Unmixing Label\hUH\images\cm_combined_10.npy')
saturated_combined_loaded = np.load('D:\Hierarchical Unmixing Label\hUH\images\saturated_combined_10.npy')
saturated_combined_loaded = np.asarray(saturated_combined_loaded, dtype=bool)

rgb_combined = plot_cm_rgb_composite(cm_combined_loaded, "cm_combined_loaded")

In [ ]:
grizzlybay_cm_norm = np.load(r'D:\Hierarchical Unmixing Label\hUH\images\grizzlybay_2024-09-30T18-25-29Z-l1a_cm_norm.npy')
victoriaLand_cm_norm = np.load(r'D:\Hierarchical Unmixing Label\hUH\images\victoriaLand_2025-01-03T13-05-25Z-l1a_cm_norm.npy')
catala_cm_norm = np.load(r'D:\Hierarchical Unmixing Label\hUH\images\catala_2025-01-28T19-17-32Z-l1a_cm_norm.npy')
menindee_cm_norm = np.load(r'D:\Hierarchical Unmixing Label\hUH\images\menindee_2025-01-21T00-00-40Z-l1a_cm_norm.npy')
deepbay_cm_norm = np.load(r'D:\Hierarchical Unmixing Label\hUH\images\deepbay_2025-01-24T18-21-34Z-l1a_cm_norm.npy')
kemigawa_cm_norm = np.load(r'D:\Hierarchical Unmixing Label\hUH\images\kemigawa_2025-01-30T01-03-42Z-l1a_cm_norm.npy')
chad_cm_norm = np.load(r'D:\Hierarchical Unmixing Label\hUH\images\chad_2025-01-30T08-58-47Z-l1a_cm_norm.npy')
mjosa_cm_norm = np.load(r'D:\Hierarchical Unmixing Label\hUH\images\mjosa_2025-01-30T10-19-48Z-l1a_cm_norm.npy')
chetumalbay_cm_norm = np.load(r'D:\Hierarchical Unmixing Label\hUH\images\chetumalbay_2025-01-17T16-01-46Z-l1a_cm_norm.npy')
lacrau_cm_norm = np.load(r'D:\Hierarchical Unmixing Label\hUH\images\lacrau_2025-01-17T09-40-10Z-l1a_cm_norm.npy')

rgb_grizzlybay = plot_cm_rgb_composite(grizzlybay_cm_norm, "grizzlybay")
rgb_victoriaLand = plot_cm_rgb_composite(victoriaLand_cm_norm, "victoriaLand")
rgb_catala = plot_cm_rgb_composite(catala_cm_norm, "catala")
rgb_menindee = plot_cm_rgb_composite(menindee_cm_norm, "menindee")
rgb_deepbay = plot_cm_rgb_composite(deepbay_cm_norm, "deepbay")
rgb_kemigawa = plot_cm_rgb_composite(kemigawa_cm_norm, "kemigawa")
rgb_chad = plot_cm_rgb_composite(chad_cm_norm, "chad")
rgb_mjosa = plot_cm_rgb_composite(mjosa_cm_norm, "mjosa")
rgb_chetumalbay = plot_cm_rgb_composite(chetumalbay_cm_norm, "chetumalbay")
rgb_lacrau = plot_cm_rgb_composite(lacrau_cm_norm, "lacrau")

### L1B DATA

In [ ]:
combined_10_v2_L1B = np.load('D:\Hierarchical Unmixing Label\hUH\images\combined_10_v2_L1B.npy')

In [ ]:
rgb_combined_10_v2_L1B = plot_cm_rgb_composite(combined_10_v2_L1B, "combined_10_v2_L1B")

In [ ]:
l1b_yucatan2_2025_02_06         = np.load(r'D:\Hierarchical Unmixing Label\hUH\images\yucatan2_2025-02-06T16-01-18Z-l1a_cm_l1b.npy')
l1b_kemigawa_2024_12_17         = np.load(r'D:\Hierarchical Unmixing Label\hUH\images\kemigawa_2024-12-17T01-01-32Z-l1a_cm_l1b.npy')
l1b_chapala_2025_02_24          = np.load(r'D:\Hierarchical Unmixing Label\hUH\images\chapala_2025-02-24T16-52-47Z-l1a_cm_l1b.npy')
l1b_grizzlybay_2025_01_27       = np.load(r'D:\Hierarchical Unmixing Label\hUH\images\grizzlybay_2025-01-27T18-19-56Z-l1a_cm_l1b.npy')
l1b_victoriaLand_2025_02_07     = np.load(r'D:\Hierarchical Unmixing Label\hUH\images\victoriaLand_2025-02-07T20-35-33Z-l1a_cm_l1b.npy')
l1b_catala_2025_01_28           = np.load(r'D:\Hierarchical Unmixing Label\hUH\images\catala_2025-01-28T19-17-32Z-l1a_cm_l1b.npy')
l1b_khnifiss_2025_02_12         = np.load(r'D:\Hierarchical Unmixing Label\hUH\images\khnifiss_2025-02-12T11-05-35Z-l1a_cm_l1b.npy')
l1b_menindee_2025_02_18         = np.load(r'D:\Hierarchical Unmixing Label\hUH\images\menindee_2025-02-18T00-10-42Z-l1a_cm_l1b.npy')
l1b_falklandsatlantic_2024_12_18= np.load(r'D:\Hierarchical Unmixing Label\hUH\images\falklandsatlantic_2024-12-18T13-25-18Z-l1a_cm_l1b.npy')
l1b_tampa_2024_11_12            = np.load(r'D:\Hierarchical Unmixing Label\hUH\images\tampa_2024-11-12T15-31-55Z-l1a_cm_l1b.npy')
# l1b_aquawatchmoreton_2024_09_02 = np.load(r'D:\Hierarchical Unmixing Label\hUH\images\aquawatchmoreton_2024-09-02T23-10-28Z-l1a_cm_l1b.npy')
# l1b_mjosa_2025_02_11            = np.load(r'D:\Hierarchical Unmixing Label\hUH\images\mjosa_2025-02-11T09-56-45Z-l1a_cm_l1b.npy')

In [ ]:
rgb_l1b_yucatan2_2025_02_06 = plot_cm_rgb_composite(l1b_yucatan2_2025_02_06, "yucatan2_2025_02_06")
rgb_l1b_kemigawa_2024_12_17 = plot_cm_rgb_composite(l1b_kemigawa_2024_12_17, "kemigawa_2024_12_17")
rgb_l1b_chapala_2025_02_24 = plot_cm_rgb_composite(l1b_chapala_2025_02_24, "chapala_2025_02_24")
rgb_l1b_grizzlybay_2025_01_27 = plot_cm_rgb_composite(l1b_grizzlybay_2025_01_27, "grizzlybay_2025_01_27")
rgb_l1b_victoriaLand_2025_02_07 = plot_cm_rgb_composite(l1b_victoriaLand_2025_02_07, "victoriaLand_2025_02_07")
rgb_l1b_catala_2025_01_28 = plot_cm_rgb_composite(l1b_catala_2025_01_28, "catala_2025_01_28")
rgb_l1b_khnifiss_2025_02_12 = plot_cm_rgb_composite(l1b_khnifiss_2025_02_12, "khnifiss_2025_02_12")
rgb_l1b_menindee_2025_02_18 = plot_cm_rgb_composite(l1b_menindee_2025_02_18, "menindee_2025_02_18")
rgb_l1b_falklandsatlantic_2024_12_18 = plot_cm_rgb_composite(l1b_falklandsatlantic_2024_12_18, "falklandsatlantic_2024_12_18")
rgb_l1b_tampa_2024_11_12 = plot_cm_rgb_composite(l1b_tampa_2024_11_12, "tampa_2024_11_12")
# rgb_l1b_aquawatchmoreton_2024_09_02 = plot_cm_rgb_composite(l1b_aquawatchmoreton_2024_09_02, "aquawatchmoreton_2024_09_02")
# rgb_l1b_mjosa_2025_02_11 = plot_cm_rgb_composite(l1b_mjosa_2025_02_11, "mjosa_2025_02_11")

In [ ]:
saturated_combined_10_v2_L1B = np.load('D:\Hierarchical Unmixing Label\hUH\images\saturated_combined_10_v2_L1B.npy')

In [ ]:
l1b_yucatan2_2025_02_06_sat         = np.load(r'D:\Hierarchical Unmixing Label\hUH\images\yucatan2_2025-02-06T16-01-18Z-l1a_saturated_l1b.npy')
l1b_kemigawa_2024_12_17_sat         = np.load(r'D:\Hierarchical Unmixing Label\hUH\images\kemigawa_2024-12-17T01-01-32Z-l1a_saturated_l1b.npy')
l1b_chapala_2025_02_24_sat          = np.load(r'D:\Hierarchical Unmixing Label\hUH\images\chapala_2025-02-24T16-52-47Z-l1a_saturated_l1b.npy')
l1b_grizzlybay_2025_01_27_sat       = np.load(r'D:\Hierarchical Unmixing Label\hUH\images\grizzlybay_2025-01-27T18-19-56Z-l1a_saturated_l1b.npy')
l1b_victoriaLand_2025_02_07_sat     = np.load(r'D:\Hierarchical Unmixing Label\hUH\images\victoriaLand_2025-02-07T20-35-33Z-l1a_saturated_l1b.npy')
l1b_catala_2025_01_28_sat           = np.load(r'D:\Hierarchical Unmixing Label\hUH\images\catala_2025-01-28T19-17-32Z-l1a_saturated_l1b.npy')
l1b_khnifiss_2025_02_12_sat         = np.load(r'D:\Hierarchical Unmixing Label\hUH\images\khnifiss_2025-02-12T11-05-35Z-l1a_saturated_l1b.npy')
l1b_menindee_2025_02_18_sat         = np.load(r'D:\Hierarchical Unmixing Label\hUH\images\menindee_2025-02-18T00-10-42Z-l1a_saturated_l1b.npy')
l1b_falklandsatlantic_2024_12_18_sat= np.load(r'D:\Hierarchical Unmixing Label\hUH\images\falklandsatlantic_2024-12-18T13-25-18Z-l1a_saturated_l1b.npy')
l1b_tampa_2024_11_12_sat            = np.load(r'D:\Hierarchical Unmixing Label\hUH\images\tampa_2024-11-12T15-31-55Z-l1a_saturated_l1b.npy')
# l1b_aquawatchmoreton_2024_09_02_sat = np.load(r'D:\Hierarchical Unmixing Label\hUH\images\aquawatchmoreton_2024-09-02T23-10-28Z-l1a_saturated_l1b.npy')
# l1b_mjosa_2025_02_11_sat            = np.load(r'D:\Hierarchical Unmixing Label\hUH\images\mjosa_2025-02-11T09-56-45Z-l1a_saturated_l1b.npy')

### L1D + MACHI DATA

In [ ]:
combined_10_L1D_112_MACHI = np.load('D:\Hierarchical Unmixing Label\hUH\images\combined_10_L1D_112_MACHI.npy')
saturated_combined_10_L1D_112_MACHI = np.load('D:\Hierarchical Unmixing Label\hUH\images\saturated_combined_10_L1D_112_MACHI.npy')

## TRAINING

### Train V1

In [ ]:
#Random init and stabelize the network 700min
DEH_function = deh.DEH(no_negative_residuals=True)
save_name = "save/dehx_10img_8end_10runs_fast_stabelized"
#DEH_combined, DEH_combined_stabelized = DEH_init_stabelize(DEH_input=DEH_function,  cm_input=cm_combined_loaded, saturated_input=saturated_combined_loaded, save_name=save_name, n_endmembers=8, n_runs=10)

In [ ]:
catala_cm_norm = np.load(r'D:\Hierarchical Unmixing Label\hUH\images\catala_2025-01-28T19-17-32Z-l1a_cm_norm.npy')
catala_saturated = np.load(r'D:\Hierarchical Unmixing Label\hUH\images\catala_2025-01-28T19-17-32Z-l1a_saturated.npy')
rgb_catala = plot_cm_rgb_composite(catala_cm_norm, "catala")
catala_save_name = "save/DEH_catala_16end_10runs_fast_stabelized"
#DEH_catala=deh.DEH(no_negative_residuals=True)
#DEH_catala, DEH_catala_stabelized = deh_init_stabelize(DEH_input=DEH_catala, cm_input=catala_cm_norm, saturated_input=catala_saturated, save_name=catala_save_name, n_endmembers=16, n_runs=10)

### Train V2

In [ ]:
DEH_V2= deh.DEH(no_negative_residuals=True)
save_name_V2 = "save/L1B_10img_16end_random"
verbose=True
#Init DEH
DEH_V2.splitting_size=1000
DEH_V2.max_depth=2
DEH_V2.max_iter=5
DEH_V2.max_nodes=2


cube=combined_10_v2_L1B.reshape(-1,1092,112)
#these set the size of the image to be plotted
DEH_V2.plot_size = (cube.shape[0],cube.shape[1])
DEH_V2.plot_aspect = 0.1

# these set the normaliztion on the data, but right now it is turned off and replaced with a nearest-neighbor noise estimate
DEH_V2.weight_power=0
DEH_V2.eps = 0.00
#dehx.wf = lambda x: (np.sum(x**2, axis=-1))**(-1/2+1/(2+dehx.eps))

#this switches between Archetype Analysis and Pure Pixel Analysis
DEH_V2.aa = False

#this is currently replaced by "mpp_tol" in the training code
DEH_V2.mixed_pix = 0

# this is the prefactor on the L2 regularization
DEH_V2.reg=0

# if this is anything other than zero, it introduces L2 regularization on the trained weights 
DEH_V2.set_mu(0)

# this turns on a sort of protection to keep endmembers from vanishing. Generally unneeded with new training
DEH_V2.use_bonus_boost = False

# turns on normalization of the data. Generally always used
DEH_V2.use_norm(True)

# if greater than 1, turns on a sort of spectral sampling with PAA. Occasionally gives good performance
# but I generally keep it at 1 to turn it off
DEH_V2.PAA_backcount = 1

# changes the gradient descent learning rate as a proportion of the optimal rate. I recommend keeping it at 1
DEH_V2.a_speed= 1.0

#Verbose print
if verbose:
    print("cm_input.shape: ", combined_10_v2_L1B.shape)
    print("cube.shape: ", cube.shape)
    print("DEH_V2.plot_size", DEH_V2.plot_size)
    print("saturated_input.shape", saturated_combined_10_v2_L1B.shape)
    

In [ ]:
#random Init
if verbose:
    print("Quick nn")
DEH_V2.neighbors = deh.quick_nn(combined_10_v2_L1B.reshape(DEH_V2.plot_size + (-1,)), k_size=1).flatten()
if verbose: 
    print("Setting neighbor weights")
DEH_V2.set_neighbor_weights(combined_10_v2_L1B)
if verbose:
    print("Random init")
DEH_V2.random_init(combined_10_v2_L1B, 16)
if verbose:
    print("Simple predict")
deh_v2_pred=DEH_V2.simple_predict(combined_10_v2_L1B)
if verbose:
    print("Getting trimmed network")

### Random training 256 endmembers

In [ ]:
save_name = "save/L1D_112_MACHI_10img_256end_random"
verbose=True
flat=combined_10_L1D_112_MACHI
cube=flat.reshape(-1,1092,112)
saturated=saturated_combined_10_L1D_112_MACHI


#Init DEH
DEH_V3= deh.DEH(no_negative_residuals=True)
DEH_V3.splitting_size=1000
DEH_V3.max_depth=2
DEH_V3.max_iter=5
DEH_V3.max_nodes=2

#these set the size of the image to be plotted
DEH_V3.plot_size = (cube.shape[0],cube.shape[1])
DEH_V3.plot_aspect = 0.1

# these set the normaliztion on the data, but right now it is turned off and replaced with a nearest-neighbor noise estimate
DEH_V3.weight_power=0
DEH_V3.eps = 0.00
#dehx.wf = lambda x: (np.sum(x**2, axis=-1))**(-1/2+1/(2+dehx.eps))

#this switches between Archetype Analysis and Pure Pixel Analysis
DEH_V3.aa = False

#this is currently replaced by "mpp_tol" in the training code
DEH_V3.mixed_pix = 0

# this is the prefactor on the L2 regularization
DEH_V3.reg=0

# if this is anything other than zero, it introduces L2 regularization on the trained weights 
DEH_V3.set_mu(0)

# this turns on a sort of protection to keep endmembers from vanishing. Generally unneeded with new training
DEH_V3.use_bonus_boost = False

# turns on normalization of the data. Generally always used
DEH_V3.use_norm(True)

# if greater than 1, turns on a sort of spectral sampling with PAA. Occasionally gives good performance
# but I generally keep it at 1 to turn it off
DEH_V3.PAA_backcount = 1

# changes the gradient descent learning rate as a proportion of the optimal rate. I recommend keeping it at 1
DEH_V3.a_speed= 1.0

#Verbose print
if verbose:
    print("cm_input.shape: ", flat.shape)
    print("cube.shape: ", cube.shape)
    print("DEH_V3.plot_size", DEH_V3.plot_size)
    print("saturated_input.shape", saturated.shape)

#random Init
if verbose:
    print("Quick nn")
DEH_V3.neighbors = deh.quick_nn(flat.reshape(DEH_V3.plot_size + (-1,)), k_size=1).flatten()
if verbose: 
    print("Setting neighbor weights")
# DEH_V3.set_neighbor_weights(flat)
DEH_V3.set_neighbor_weights_memory_efficient(flat)
if verbose:
    print("Random init")
DEH_V3.random_init(flat, 256, seed=np.random.randint(0, 1000))
if verbose:
    print("Simple predict")
DEH_V3_pred=DEH_V3.simple_predict(flat)

frontier_nodes = tree_walk(DEH_V3, n=8, verbose=True)
print(f"Frontier nodes: {frontier_nodes}")

    



In [ ]:
#del DEH_V3_pred
del DEH_V3.nodes
del DEH_V3
gc.collect()

In [ ]:
DEH_V3.save(save_name + '.h5')

In [ ]:
# Test the pruning function on DEH_V3_LOAD with the frontier nodes
prune_deh_obj(DEH_V3, frontier_nodes)

In [ ]:
# Print nodes by depth for the original and pruned DEH objects
print("\nPruned DEH object:")
print_nodes_by_depth(DEH_V3)

In [ ]:
#Stabelized
# DEH_V3_pred=DEH_V3_LOAD.simple_predict(combined_10_v2_L1B)
# DEH_V3_LOAD.plot_size=(5980,1092)
# verbose=False
#DEH_V3_LOAD_stab=DEH_V3_LOAD.get_trimmed_network(DEH_V3_LOAD.nodes) #Can replace with copy of DEH_input instead of get trimmed network since it acceptes all nodes
# if verbose:
print("Quick nn")
DEH_V3.neighbors = deh.quick_nn(flat.reshape(DEH_V3.plot_size + (-1,)), k_size=1).flatten()
# if verbose:
print("Setting neighbor weights")

DEH_V3.set_neighbor_weights_memory_efficient(flat)
# if verbose:
print("DEH_stabelized.full_weights.shape",DEH_V3.full_weights.shape)
#saturated_input = np.asarray(saturated_input, dtype=bool)
DEH_V3.full_weights[saturated]=0
DEH_V3.PAA_backcount=1

# if verbose:
print("Accepted network stablization")
DEH_V3.accepted_network_stablization(flat, n_runs=10, n_pts=(1000,10000), obj_record=(), sampling_points=(), mpp_tol=0.2, step_delta=0.05, reg_max=0.2, name=save_name)

## Plotting

In [ ]:
plot_level_and_overlay(r'D:\Hierarchical Unmixing Label\hUH\save\dehx_10img_8end_10runs_fast_stabelized_aa_FINAL.h5', lacrau_cm_norm, plot_only_last_level=True)

In [ ]:
plot_level_and_overlay(r'D:\Hierarchical Unmixing Label\hUH\save\dehx_10img_8end_10runs_fast_stabelized_aa_FINAL.h5', kemigawa_cm_norm, plot_only_last_level=True)

In [ ]:
plot_level_and_overlay(r'D:\Hierarchical Unmixing Label\hUH\save\dehx_10img_8end_10runs_fast_stabelized_aa_FINAL.h5', grizzlybay_cm_norm, plot_only_last_level=True)

In [ ]:
plot_spectra_for_level(r'D:\Hierarchical Unmixing Label\hUH\save\dehx_10img_8end_10runs_fast_stabelized_aa_FINAL.h5', lacrau_cm_norm, level_input=1)

In [ ]:
plot_level_and_overlay(r'D:\Hierarchical Unmixing Label\hUH\save\DEH_catala_16end_10runs_fast_stabelized_aa_FINAL.h5',catala_cm_norm, plot_only_last_level=True )

In [ ]:
plot_level_and_overlay(r'D:\Hierarchical Unmixing Label\hUH\save\DEH_catala_16end_10runs_fast_stabelized_aa_FINAL.h5',catala_cm_norm, plot_only_last_level=True, binarize=True, opacity=.5)

## Testing

### TEST 1

In [ ]:
DEH_SNOW= deh.DEH(no_negative_residuals=True)
DEH_SNOW.load(r'D:\Hierarchical Unmixing Label\hUH\save\DEH_catala_16end_10runs_fast_stabelized_aa_FINAL.h5')
pred=DEH_SNOW.simple_predict(catala_cm_norm)
DEH_SNOW_binarized=copy.deepcopy(DEH_SNOW)
DEH_SNOW_binarized.binarize_lmdas()
DEH_SNOW_binarized.lmda_2_map()
DEH_SNOW_binarized.display_level(1)

In [ ]:
binarized_model = copy.deepcopy(DEH_SNOW)
save_binarized_labels_to_npy(binarized_model, catala_cm_norm, "catala_binary_labels.npy")

In [ ]:
catala_cm_norm = np.load(r'D:\Hierarchical Unmixing Label\hUH\images\catala_2025-01-28T19-17-32Z-l1a_cm_norm.npy')
rgb_catala = plot_cm_rgb_composite(catala_cm_norm, "catala")
plot_level_labels("save\catala_binary_labels.npy", level=1)
plot_level_labels("save\catala_binary_labels.npy", level=2)
plot_level_labels("save\catala_binary_labels.npy", level=3)
plot_level_labels("save\catala_binary_labels.npy", level=4)
plot_level_labels("save\catala_binary_labels.npy", level=5)
plot_level_labels("save\catala_binary_labels.npy", level=6)

### TEST 2


In [ ]:
DEH_V2_grizzly=copy.deepcopy(DEH_V2)
DEH_V2_grizzly.plot_size = (598,1092)


In [ ]:
DEH_V2_grizzly.simple_predict(l1b_grizzlybay_2025_01_27)
plt.figure(figsize=(8,2))
plt.axis('off')
plt.imshow(rgb_l1b_grizzlybay_2025_01_27, aspect=0.1)
DEH_V2_grizzly.display_level(1)
DEH_V2_grizzly.display_level(6)

In [ ]:
DEH_V2_falklands=copy.deepcopy(DEH_V2)
DEH_V2_falklands.plot_size = (598,1092)

In [ ]:
DEH_V2_falklands.simple_predict(l1b_falklandsatlantic_2024_12_18)
plt.figure(figsize=(8,2))
plt.axis('off')
plt.imshow(rgb_l1b_falklandsatlantic_2024_12_18, aspect=0.1)
DEH_V2_falklands.display_level(1)
DEH_V2_falklands.display_level(6)

### TEST REDUCE AND STABELIZE 

In [ ]:
combined_10_L1D_112_MACHI = np.load('D:\Hierarchical Unmixing Label\hUH\images\combined_10_L1D_112_MACHI.npy')
saturated_combined_10_L1D_112_MACHI = np.load('D:\Hierarchical Unmixing Label\hUH\images\saturated_combined_10_L1D_112_MACHI.npy')
flat=combined_10_L1D_112_MACHI
saturated=saturated_combined_10_L1D_112_MACHI
DEH_V3_LOAD= deh.DEH(no_negative_residuals=True)
DEH_V3_LOAD.load(r'D:\Hierarchical Unmixing Label\hUH\save\L1D_112_MACHI_10img_256end_random.h5')
DEH_V3_LOAD.verbose = True
DEH_V3_pred=DEH_V3_LOAD.simple_predict(flat)

In [ ]:
# Test the tree walk function on DEH_V3_LOAD
frontier_nodes = tree_walk(DEH_V3_LOAD, n=8)
print(frontier_nodes)

In [ ]:
# Test the pruning function on DEH_V3_LOAD with the frontier nodes
prune_deh_obj(DEH_V3_LOAD, frontier_nodes)

In [ ]:
# Print nodes by depth for the original and pruned DEH objects
print("\nPruned DEH object:")
print_nodes_by_depth(DEH_V3_LOAD)

In [ ]:
#Stabelized
# DEH_V3_pred=DEH_V3_LOAD.simple_predict(combined_10_v2_L1B)
# DEH_V3_LOAD.plot_size=(5980,1092)
# verbose=False
#DEH_V3_LOAD_stab=DEH_V3_LOAD.get_trimmed_network(DEH_V3_LOAD.nodes) #Can replace with copy of DEH_input instead of get trimmed network since it acceptes all nodes
# if verbose:
print("Quick nn")
DEH_V3_LOAD.neighbors = deh.quick_nn(flat.reshape(DEH_V3_LOAD.plot_size + (-1,)), k_size=1).flatten()
# if verbose:
print("Setting neighbor weights")

DEH_V3_LOAD.set_neighbor_weights(flat)
# if verbose:
print("DEH_stabelized.full_weights.shape",DEH_V3_LOAD.full_weights.shape)
#saturated_input = np.asarray(saturated_input, dtype=bool)
DEH_V3_LOAD.full_weights[saturated]=0
DEH_V3_LOAD.PAA_backcount=1

save_name="save\MACHI_10img_256to8_stab.h5"
# if verbose:
print("Accepted network stablization")
DEH_V3_LOAD.accepted_network_stablization(flat, n_runs=10, n_pts=(1000,10000), obj_record=(), sampling_points=(), mpp_tol=0.2, step_delta=0.05, reg_max=0.2, name=save_name)

* test reglectance data that is scaled to 0-1? may be neccesary for DEH
* test this to see if sparsift works better then
* work out the sparse tree function
* speed up trainnig, sparsify sweep loop to slow? faster iteration? 
* stuck at around .8 but threshold is at .2
* look into the traditional training sparse tree output
* better prints form accepted stabilization
* better naming 

After training
* data should be in range 0-1, else errors happens
* sparsify sweep does not need step of 0.01 or 20 steps. very slow
* lines in data, will see if persists in binarysation, might be a problem with the data, since it is radiance based not reflectance
* optimally is should be TOA with atmosphere correction-> bottom of atmosphere reflection
* 

### TEST 4

In [ ]:
combined_10_v2_MACHI = np.load('D:\Hierarchical Unmixing Label\hUH\images\combined_10_v2_MACHI.npy')
DEH_v3_aa= deh.DEH(no_negative_residuals=True)
DEH_v3_aa.load(r'D:\Hierarchical Unmixing Label\hUH\save\MACHI_10img_256to8_stab.h5_aa_FINAL.h5')
DEH_v3_aa_pred=DEH_v3_aa.simple_predict(combined_10_v2_MACHI)
DEH_v3_aa.binarize_lmdas()
DEH_v3_aa.lmda_2_map()
DEH_v3_aa.display_level(1)

In [ ]:
save_binarized_labels_to_npy(DEH_v3_aa, combined_10_v2_MACHI, "save\combined_MACHI_binary_labels.npy")

### TEST 5

In [ ]:
aregantsea2_2025_03_11_MACHI = np.load(r'D:\Hierarchical Unmixing Label\hUH\images\aregantsea2_2025-03-11T08-12-43Z-l1a_cm_machi.npy')

In [ ]:

DEH_v3_aa= deh.DEH(no_negative_residuals=True)
DEH_v3_aa.load(r'D:\Hierarchical Unmixing Label\hUH\save\MACHI_10img_256to8_stab.h5_aa_FINAL.h5')
DEH_v3_aa.plot_size=(598,1092)
DEH_v3_aa_pred=DEH_v3_aa.simple_predict(aregantsea2_2025_03_11_MACHI)
DEH_v3_aa.display_level(4)
DEH_v3_aa.binarize_lmdas()
DEH_v3_aa.lmda_2_map()
DEH_v3_aa.display_level(4)

In [ ]:
save_binarized_labels_to_npy(DEH_v3_aa, aregantsea2_2025_03_11_MACHI, r"save\aregantsea2_2025_03_11_MACHI_binary_labels.npy")

In [ ]:
losmanzanosfire_2025_03_11_MACHI = np.load(r'D:\Hierarchical Unmixing Label\hUH\images\losmanzanosfire_2025-03-11T14-56-42Z-l1a_cm_MACHI.npy')

In [ ]:

DEH_v3_aa= deh.DEH(no_negative_residuals=True)
DEH_v3_aa.load(r'D:\Hierarchical Unmixing Label\hUH\save\MACHI_10img_256to8_stab.h5_aa_FINAL.h5')
DEH_v3_aa.plot_size=(598,1092)
DEH_v3_aa_pred=DEH_v3_aa.simple_predict(losmanzanosfire_2025_03_11_MACHI)
DEH_v3_aa.display_level(4)
DEH_v3_aa.binarize_lmdas()
DEH_v3_aa.lmda_2_map()
DEH_v3_aa.display_level(4)

In [ ]:
save_binarized_labels_to_npy(DEH_v3_aa, losmanzanosfire_2025_03_11_MACHI, r"save\losmanzanosfire_2025_03_11_MACHI_binary_labels.npy")

### TEST L1B

In [ ]:
aregantsea2_2025_03_11 = np.load(r'D:\Hierarchical Unmixing Label\hUH\images\aregantsea2_2025-03-11T08-12-43Z-l1a_cm_L1B.npy')
aregantsea2_2025_03_11_saturated = np.load(r'D:\Hierarchical Unmixing Label\hUH\images\aregantsea2_2025-03-11T08-12-43Z-l1a_saturated_L1B.npy')
save_name="save/L1B_1img_256end_random.h5"
DEH_load="D:\Hierarchical Unmixing Label\hUH\save\L1B_1img_256end_random.h5"
labels_path="save\L1B_1img_8end_stabelized_labels"
stabelized_path="save\L1B_1img_8end_stabelized_ppa.h5"

In [ ]:
train_random_DEH(aregantsea2_2025_03_11,endmembers=256,save_name=save_name,verbose=True)

In [ ]:
pruned_path=prune_DEH(save_name,image_data=aregantsea2_2025_03_11,endmembers=8,verbose=True)

In [ ]:
stabelize_DEH(DEH_path=pruned_path, image_data=aregantsea2_2025_03_11, saturated_image_data=aregantsea2_2025_03_11_saturated, n_runs=10, step_delta=0.05, verbose=False)

In [ ]:
save_binarized_labels_to_npy(stabelized_path, aregantsea2_2025_03_11)

### L1D 112

In [ ]:
combined_10_L1D_112_MACHI = np.load('D:\Hierarchical Unmixing Label\hUH\images\combined_10_L1D_112_MACHI.npy')
saturated_combined_10_L1D_112_MACHI = np.load('D:\Hierarchical Unmixing Label\hUH\images\saturated_combined_10_L1D_112_MACHI.npy')

flat=combined_10_L1D_112_MACHI
saturated=saturated_combined_10_L1D_112_MACHI
stabelized_path='save\L1D_112_MACHI_10img_8end_stabelized_aa.h5'

In [ ]:
tampa_2024_11_12_L1D_112=np.load(r'images\tampa_2024-11-12T15-31-55Z-l1a_cm_L1D_112_MACHI.npy')
tampa_2024_11_12_L1D_112_save=r'save\tampa_2024_11_12_L1D_112_labels'
save_binarized_labels_to_npy(stabelized_path, tampa_2024_11_12_L1D_112, tampa_2024_11_12_L1D_112_save)

In [ ]:
save_binarized_labels_to_npy(stabelized_path, combined_10_L1D_112_MACHI)

In [ ]:

DEH_L1D= deh.DEH(no_negative_residuals=True)
DEH_L1D.load(r'D:\Hierarchical Unmixing Label\hUH\save\L1D_112_MACHI_10img_8end_stabelized_aa.h5')
DEH_L1D.verbose = True
DEH_L1D_pred=DEH_L1D.simple_predict(flat)

### L1A 120

In [ ]:
stabelized_path='save\L1D_112_MACHI_10img_8end_stabelized_aa.h5'
image = np.load(r'images\tampa_2024-11-12T15-31-55Z-l1a_cm_L1D_112_MACHI.npy')
save_path=r'save\tampa_2024_11_12_L1A_120_labels'
save_binarized_labels_to_npy(stabelized_path, image, save_path)